label encoded geography

Data Preporcessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv(r"C:\Users\okechukwu chude\Documents\Neural Network\Customer Churn\Churn_Modelling.csv")
x = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
print(dataset.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

Encoding categorical data


In [4]:
# Label Encoding the "geography" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 1] = le.fit_transform(x[:, 1])
print(x)

[[619 0 'Female' ... 1 1 101348.88]
 [608 2 'Female' ... 0 1 112542.58]
 [502 0 'Female' ... 1 0 113931.57]
 ...
 [709 0 'Female' ... 0 1 42085.58]
 [772 1 'Male' ... 1 0 92888.52]
 [792 0 'Female' ... 1 0 38190.78]]


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])
print(x)

[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]


In [6]:
# Get the first two rows
first_two_rows = x[:2]

print(first_two_rows)

[[619 0 0 42 2 0.0 1 1 1 101348.88]
 [608 2 0 41 1 83807.86 1 0 1 112542.58]]


Splitting into training and test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Developing the ANN

In [9]:
from scikeras.wrappers import KerasClassifier

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
from keras.models import Sequential #required to initialise ann
from keras.layers import Dense # required to build laters of ann

In [12]:
#function to build ANN architecture
def build_classifier():
    #initialise
    classifier = Sequential()
    #add input layer and first hidden layer
    classifier.add(Dense(units=6, activation='relu', input_dim=10))
    #add second hidden layer
    classifier.add(Dense(units=6, activation='relu'))
    #add output layer
    classifier.add(Dense(units=1, activation='sigmoid'))
    #compile ANN
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier


In [13]:
#create global classifier variable
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [14]:
mean = accuracies.mean()
variance = accuracies.std()
print('mean accuracy:', mean , 'accuracy variance:', variance)

mean accuracy: 0.8568749999999999 accuracy variance: 0.012553012586626363


Handling Overfitting

In [15]:
from keras.layers import Dropout #to prevent overfitting

In [49]:
classifier = Sequential()

#add input layer and first hidden layer
classifier.add(Dense(units=6, activation='relu', input_dim=10))
classifier.add(Dropout(rate = 0.1))

#add second hidden layer
classifier.add(Dense(units=6, activation='relu'))
classifier.add(Dropout(rate = 0.1))

#add output layer
classifier.add(Dense(units=1, activation='sigmoid'))

#compile ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

Tuning the ANN using GridSearch

In [17]:
from sklearn.model_selection import GridSearchCV

In [33]:
from keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [40]:
#function to build ANN architecture
def build_classifier(optimizer='adam'):
    #initialise
    classifier = Sequential()
    
    #add input layer and first hidden layer
    classifier.add(Dense(units=6, activation='relu', input_dim=10))
    
    #add second hidden layer
    classifier.add(Dense(units=6, activation='relu'))
    
    #add output layer
    classifier.add(Dense(units=1, activation='sigmoid'))
    
    #compile ANN
    classifier.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

In [41]:
#create global classifier variable
classifier = KerasClassifier(build_fn = build_classifier)

Create Dictionary that contains the Hyper parameters

In [42]:
parameters = {'optimizer': ['adam', 'rmsprop'],
            'batch_size' :[25, 32],
             'epochs': [100, 500]
             }

In [43]:
grid_search = GridSearchCV(estimator= classifier,
                           param_grid= parameters,
                           scoring= 'accuracy',
                           cv = 10)

In [48]:
#fit gridseasrch to training set
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100


C:\Users\okechukwu chude\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


288/288 [==============================] - 2s 3ms/step - loss: 0.5622 - accuracy: 0.7490
Epoch 2/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4655 - accuracy: 0.7961
Epoch 3/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4484 - accuracy: 0.7967
Epoch 4/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4408 - accuracy: 0.7983
Epoch 5/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4356 - accuracy: 0.8011
Epoch 6/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4309 - accuracy: 0.8037
Epoch 7/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4249 - accuracy: 0.8051
Epoch 8/100
288/288 [==============================] - 1s 2ms/step - loss: 0.4178 - accuracy: 0.8078
Epoch 9/100
288/288 [==============================] - 1s 3ms/step - loss: 0.4082 - accuracy: 0.8139
Epoch 10/100
288/288 [==============================] - 1s 2ms/step - loss: 0.3975 - accuracy: 0.8175
E

KeyboardInterrupt: 

In [45]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [46]:
best_parameters

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}

In [47]:
best_accuracy

0.8606250000000001